## simple but good LSTM

In [7]:
#gidi: seems like this guy first does the standard tokenizing, encoding, padding stuff
'''
Single model may achieve LB scores at around 0.29+ ~ 0.30+
Average ensembles can easily get 0.28+ or less
Don't need to be an expert of feature engineering
All you need is a GPU!!!!!!!

The code is tested on Keras 2.0.0 using Tensorflow backend, and Python 2.7

According to experiments by kagglers, Theano backend with GPU may give bad LB scores while
        the val_loss seems to be fine, so try Tensorflow backend first please
'''

########################################
## import packages
########################################
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
#from keras.layers.merge import concatenate # keras 2.0
from keras.layers import merge
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')

In [27]:
def do_clip(arr, mx):
    clipped = np.clip(arr, (1-mx)/1, mx)
    return clipped/clipped.sum(axis=1)[:, np.newaxis]

In [8]:
########################################
## set directories and parameters
########################################
BASE_DIR = '../input/'
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
TRAIN_DATA_FILE = '/mnt/quora_kaggle/train.csv'
TEST_DATA_FILE = '/mnt/quora_kaggle/test.csv'
MAX_SEQUENCE_LENGTH = 30
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = np.random.randint(175, 275)
num_dense = np.random.randint(100, 150)
rate_drop_lstm = 0.15 + np.random.rand() * 0.25
rate_drop_dense = 0.15 + np.random.rand() * 0.25

act = 'relu'
re_weight = True # whether to re-weight classes to fit the 17.5% share in test set

STAMP = 'lstm_%d_%d_%.2f_%.2f'%(num_lstm, num_dense, rate_drop_lstm, \
        rate_drop_dense)

In [9]:
'''
########################################
## index word vectors
########################################
print('Indexing word vectors')

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))
'''

"\n########################################\n## index word vectors\n########################################\nprint('Indexing word vectors')\n\nword2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,         binary=True)\nprint('Found %s word vectors of word2vec' % len(word2vec.vocab))\n"

In [10]:
########################################
## process texts in datasets
########################################
print('Processing text dataset')

# The function "text_to_wordlist" is from
# https://www.kaggle.com/currie32/quora-question-pairs/the-importance-of-cleaning-text
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

texts_1 = [] 
texts_2 = []
labels = []
with codecs.open(TRAIN_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        texts_1.append(text_to_wordlist(values[3]))
        texts_2.append(text_to_wordlist(values[4]))
        labels.append(int(values[5]))
print('Found %s texts in train.csv' % len(texts_1))

test_texts_1 = []
test_texts_2 = []
test_ids = []
with codecs.open(TEST_DATA_FILE, encoding='utf-8') as f:
    reader = csv.reader(f, delimiter=',')
    header = next(reader)
    for values in reader:
        test_texts_1.append(text_to_wordlist(values[1]))
        test_texts_2.append(text_to_wordlist(values[2]))
        test_ids.append(values[0])
print('Found %s texts in test.csv' % len(test_texts_1))



Processing text dataset
Found 404290 texts in train.csv
Found 2345796 texts in test.csv


In [12]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS) #nb_words in keras 2.0
tokenizer.fit_on_texts(texts_1 + texts_2 + test_texts_1 + test_texts_2)

sequences_1 = tokenizer.texts_to_sequences(texts_1)
sequences_2 = tokenizer.texts_to_sequences(texts_2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts_2)

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
data_2 = pad_sequences(sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
labels = np.array(labels)
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)

test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_ids = np.array(test_ids)

Found 120499 unique tokens
Shape of data tensor: (404290, 30)
Shape of label tensor: (404290,)


In [13]:
#gidi: my w2v
import bcolz
import pickle
import re
def load_array(fname):
    return bcolz.open(fname)[:]

def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'),encoding='latin1'),
        pickle.load(open(loc+'_idx.pkl','rb'),encoding='latin1'))

#load word vecs
vec_size=300 #glove has 50,100,200,300
vecs, words, wordidx = load_vectors('/mnt/glove/6B.'+str(vec_size)+'d')

In [200]:
def w2v(w): return vecs[wordidx[w]]

In [14]:
min(MAX_NB_WORDS, len(word_index))+1

120500

In [15]:
# i run my own w2v instead
embedding_matrix=vecs
nb_words=400000

In [16]:
'''
########################################
## prepare embeddings - skip that, I use my own w2v
########################################
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
'''

"\n########################################\n## prepare embeddings - skip that, I use my own w2v\n########################################\nprint('Preparing embedding matrix')\n\nnb_words = min(MAX_NB_WORDS, len(word_index))+1\n\nembedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))\nfor word, i in word_index.items():\n    if word in word2vec.vocab:\n        embedding_matrix[i] = word2vec.word_vec(word)\nprint('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))\n"

In [17]:
########################################
## sample train/validation data
########################################
#np.random.seed(1234)
perm = np.random.permutation(len(data_1))
idx_train = perm[:int(len(data_1)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data_1)*(1-VALIDATION_SPLIT)):]

data_1_train = np.vstack((data_1[idx_train], data_2[idx_train]))
data_2_train = np.vstack((data_2[idx_train], data_1[idx_train])) #gidi: he puts q1-q2, and q2-q1 what doubles the data set///
labels_train = np.concatenate((labels[idx_train], labels[idx_train]))

data_1_val = np.vstack((data_1[idx_val], data_2[idx_val]))
data_2_val = np.vstack((data_2[idx_val], data_1[idx_val]))
labels_val = np.concatenate((labels[idx_val], labels[idx_val]))

weight_val = np.ones(len(labels_val))
if re_weight:
    weight_val *= 0.472001959
    weight_val[labels_val==0] = 1.309028344

In [43]:
data_1[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    2,    3,    1, 1254,   61, 1254,
       2921,    8,  578,    7,  759,  370,    7,   35], dtype=int32)

In [20]:
########################################
## define the model structure
########################################
embedding_layer = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

lstm_layer = LSTM(num_lstm, dropout_W=rate_drop_lstm, dropout_U=rate_drop_lstm)

sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
y1 = lstm_layer(embedded_sequences_2)

merged = merge([x1, y1],mode='concat')
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

merged = Dense(num_dense, activation=act)(merged)
merged = Dropout(rate_drop_dense)(merged)
merged = BatchNormalization()(merged)

preds = Dense(1, activation='sigmoid')(merged)

In [ ]:
########################################
## add class weight
########################################
if re_weight:
    class_weight = {0: 1.309028344, 1: 0.472001959}
else:
    class_weight = None

In [42]:
labels_train

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
########################################
## train the model
########################################
results_path='/mnt/quora_kaggle/results/'

model = Model([sequence_1_input, sequence_2_input],preds)

model.compile(loss='binary_crossentropy',optimizer='nadam',metrics=['acc'])

#model.summary()
print(STAMP)

early_stopping =EarlyStopping(monitor='val_loss', patience=3)
bst_model_path = results_path+STAMP + '.h5'

model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

hist = model.fit([data_1_train, data_2_train], labels_train, \
        validation_data=([data_1_val, data_2_val], labels_val, weight_val), \
        nb_epoch=200, batch_size=2048, shuffle=True, \
        class_weight=class_weight, callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path)
bst_val_score = min(hist.history['val_loss'])

In [32]:
########################################
## make the submission
########################################
print('Start making the submission before fine-tuning')

preds = model.predict([test_data_1, test_data_2], batch_size=8192, verbose=1)
preds += model.predict([test_data_2, test_data_1], batch_size=8192, verbose=1)
preds /= 2



Start making the submission before fine-tuning
 155648/2345796 [>.............................] - ETA: 168s

KeyboardInterrupt: 

In [ ]:
#new_preds_1 = do_clip(sub_preds,0.82)

In [33]:
submission = pd.DataFrame({'test_id':test_ids, 'is_duplicate':np.clip(preds, 0.03, 0.97).ravel()})
submission.to_csv('%.4f_'%(results_path+bst_val_score)+STAMP+'.csv', index=False)

## abishak LSTM

In [204]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.engine.topology import Merge
from keras.layers import TimeDistributed, Lambda
from keras.layers import Convolution1D, GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence, text

In [207]:
#data = pd.read_csv('data/quora_duplicate_questions.tsv', sep='\t')
y = labels#data.is_duplicate.values

tk = text.Tokenizer(nb_words=200000)

max_len = 40
tk.fit_on_texts(list(data.question1.values) + list(data.question2.values.astype(str)))
x1 = tk.texts_to_sequences(data.question1.values)
x1 = sequence.pad_sequences(x1, maxlen=max_len)

x2 = tk.texts_to_sequences(data.question2.values.astype(str))
x2 = sequence.pad_sequences(x2, maxlen=max_len)

word_index = tk.word_index

ytrain_enc = np_utils.to_categorical(y)

In [208]:
'''
embeddings_index = {}
f = open('data/glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
'''
max_features = 200000
filter_length = 5
nb_filter = 64
pool_length = 4


In [214]:
#using my stuff here
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            try:
                src_idx = wordidx[word]
                emb[i] = vecs[src_idx]
            except:
                emb[i] = normal(scale=0.6, size=(n_fact,))
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [220]:
wordidx

{'biennials': 130852,
 'verplank': 42458,
 'soestdijk': 274735,
 'woode': 311324,
 'mdbo': 212156,
 'sowell': 65544,
 'mdbu': 119490,
 'woods': 2507,
 'spiders': 19780,
 'mdbs': 285106,
 'mdbr': 131910,
 'woody': 10967,
 'trawling': 56782,
 'hwasung': 389348,
 'spidery': 126257,
 'regularize': 106404,
 'hennings': 85340,
 'canes': 34403,
 'canet': 110349,
 'caney': 97434,
 'yusaf': 223713,
 'chanthaburi': 154208,
 'igual': 120122,
 'comptuex': 399730,
 'caned': 78728,
 'mirisch': 180868,
 'kalecik': 246349,
 'rickman': 54345,
 'jacquemod': 117004,
 'ioannidis': 142172,
 'canel': 299678,
 'canem': 357071,
 'afikoman': 354205,
 'draÅ¾a': 228083,
 'heliothis': 265609,
 'replacer': 292996,
 'pigment': 29943,
 'bogyman': 343214,
 'transvestism': 191391,
 'fuste': 333859,
 'Ä\x91Æ°á»\x9dng': 265245,
 'seamier': 168003,
 'illtyd': 307371,
 'wooded': 17054,
 'shipworms': 359396,
 'boorstein': 310641,
 'grueling': 17482,
 'persita': 353293,
 'wooden': 4836,
 'iÌ\x87kinci': 266703,
 'virtuosos':

In [221]:
idx2word = {v: k for k, v in wordidx.items()}
vocab_size=len(word_index)
normal=np.random.normal
emb = create_emb()


In [238]:
def add_dense_layer(model):
    model.add(Dense(300))
    model.add(PReLU())
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

In [249]:
#there is a thing here with the w2v: the author use nltk tokenizer which reduced number of words to 85K
#therefore default emb_matrix should be 85K length. however, mine 

model = Sequential()
print('Build model...')

model1 = Sequential()
model1.add(Embedding(vocab_size ,300, weights=[emb],input_length=40,trainable=False))

model1.add(TimeDistributed(Dense(300, activation='relu')))
model1.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,)))

model2 = Sequential()
model2.add(Embedding(vocab_size  ,300, weights=[emb], input_length=40,trainable=False))

model2.add(TimeDistributed(Dense(300, activation='relu')))
model2.add(Lambda(lambda x: K.sum(x, axis=1), output_shape=(300,)))

model3 = Sequential()
model3.add(Embedding(vocab_size  ,300,weights=[emb],input_length=40, trainable=False))
model3.add(Convolution1D(nb_filter=nb_filter,filter_length=filter_length, border_mode='valid',
                         activation='relu',subsample_length=1))
model3.add(Dropout(0.2))

model3.add(Convolution1D(nb_filter=nb_filter,filter_length=filter_length, border_mode='valid',activation='relu',
                         subsample_length=1))

model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.2))

model3.add(Dense(300))
model3.add(Dropout(0.2))
model3.add(BatchNormalization())

model4 = Sequential()
model4.add(Embedding(vocab_size,300, weights=[emb],input_length=40, trainable=False))
model4.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='valid',
                         activation='relu', subsample_length=1))
model4.add(Dropout(0.2))

model4.add(Convolution1D(nb_filter=nb_filter,filter_length=filter_length,border_mode='valid',  activation='relu',
                         subsample_length=1))

model4.add(GlobalMaxPooling1D())
model4.add(Dropout(0.2))

model4.add(Dense(300))
model4.add(Dropout(0.2))
model4.add(BatchNormalization())
model5 = Sequential()
model5.add(Embedding(vocab_size , 300, input_length=40, dropout=0.2))
model5.add(LSTM(300, dropout_W=0.2, dropout_U=0.2))

model6 = Sequential()
model6.add(Embedding(vocab_size, 300, input_length=40, dropout=0.2))
model6.add(LSTM(300, dropout_W=0.2, dropout_U=0.2))

merged_model = Sequential()
merged_model.add(Merge([model1, model2, model3, model4, model5, model6], mode='concat'))
merged_model.add(BatchNormalization())


for i in range(4):
    add_dense_layer(model)

merged_model.add(Dense(1))
merged_model.add(Activation('sigmoid'))

merged_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpoint = ModelCheckpoint('/mnt/quora_kaggle/results/abishek_weights.h5', monitor='val_loss'\
                             , save_best_only=True, verbose=2)



Build model...


ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

In [237]:
abi_model=merged_model

In [234]:
test_data_2.shape

(2345796, 30)

In [230]:
merged_model.fit([x1, x2, x1, x2, x1, x2], y=y, batch_size=384, nb_epoch=200,
                 verbose=1, validation_split=0.1, shuffle=True, callbacks=[checkpoint,early_stopping])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 363861 samples, validate on 40429 samples
Epoch 1/200
363861/363861 [==============================] - 378s - loss: 0.5439 - acc: 0.7195 - val_loss: 0.4634 - val_acc: 0.7689
Epoch 2/200
363861/363861 [==============================] - 372s - loss: 0.4472 - acc: 0.7839 - val_loss: 0.4512 - val_acc: 0.7815
Epoch 3/200
363861/363861 [==============================] - 372s - loss: 0.3878 - acc: 0.8177 - val_loss: 0.4221 - val_acc: 0.8008
Epoch 4/200
363861/363861 [==============================] - 370s - loss: 0.3364 - acc: 0.8457 - val_loss: 0.4319 - val_acc: 0.7956
Epoch 5/200
363861/363861 [==============================] - 370s - loss: 0.2978 - acc: 0.8654 - val_loss: 0.4427 - val_acc: 0.8039
Epoch 6/200
363861/363861 [==============================] - 370s - loss: 0.2669 - acc: 0.8809 - val_loss: 0.4675 - val_acc: 0.8038
Epoch 7/200
 17664/363861 [>.............................] - ETA: 344s - loss: 0.2271 - acc: 0.8999

KeyboardInterrupt: 

In [246]:
from sklearn.metrics import accuracy_score

In [248]:
accuracy_score(y,np.round(abi_preds))

0.90750204061441042

In [241]:
abi_preds=merged_model.predict([x1, x2, x1, x2, x1, x2])

## Level 2 Model

### combined LSTM

### feature engineering

In [191]:
from fuzzywuzzy import fuzz
import tqdm
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
from nltk.corpus import stopwords

In [201]:
def sent2vec(s):
    words = str(s).lower()#.decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(w2v[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [ ]:
    words = str(s).lower()#.decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]

In [202]:
question1_vectors = np.zeros((data.shape[0], 300))
error_count = 0
stop_words = stopwords.words('english')

for i, q in tqdm.tqdm(enumerate(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

question2_vectors  = np.zeros((data.shape[0], 300))
for i, q in tqdm.tqdm(enumerate(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)


0it [00:00, ?it/s]/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  

538it [00:00, 5375.47it/s]
1079it [00:00, 5383.88it/s]
1617it [00:00, 5381.98it/s]
2156it [00:00, 5382.77it/s]
2711it [00:00, 5429.73it/s]
3260it [00:00, 5445.77it/s]
3812it [00:00, 5467.35it/s]
4358it [00:00, 5463.56it/s]
4878it [00:00, 5381.56it/s]
5431it [00:01, 5422.80it/s]
5973it [00:01, 5420.54it/s]
6534it [00:01, 5474.87it/s]
7089it [00:01, 5494.73it/s]
7634it [00:01, 5479.96it/s]
8180it [00:01, 5470.68it/s]
8732it [00:01, 5483.37it/s]
9285it [00:01, 5494.83it/s]
9834it [00:01, 5492.30it/s]
10383it [00:01, 5424.76it/s]
10930it [00:02, 5435.60it/s]
11477it [00:02, 5445.71it/s]
12022it [00:02, 5433.57it/s]
12568it [00:02, 5439.22it/s]
13124it [00:02, 5472.81it/s]
13672it [00:02, 5471.27it/s]
14224it [00:02, 5484.35it/s]
14773it [00:02, 5479.43it/s]
15321it [00:02, 5461.83it/s]
15868it [00:02, 5434.33it/s]
16415it [00:03, 54

In [203]:
question1_vectors

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [66]:
preds_trn = model.predict([data_1_train, data_2_train], batch_size=8192, verbose=1)
preds_trn += model.predict([data_2_train, data_1_train], batch_size=8192, verbose=1)
preds_trn /= 2

727722/727722 [==============================] - 55s    


In [93]:
preds_val = model.predict([data_1_val, data_2_val], batch_size=8192, verbose=1)
preds_val += model.predict([data_2_val, data_1_val], batch_size=8192, verbose=1)
preds_val /= 2

80858/80858 [==============================] - 6s     


#### do xgboost, first only on length

In [151]:
len_q1 = pd.Series(texts_1).apply(lambda x: len(str(x)))
len_q2 = pd.Series(texts_2).apply(lambda x: len(str(x)))
diff_len = len_q1 - len_q2
len_char_q1 = pd.Series(texts_1).apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
len_char_q2 = pd.Series(texts_2).apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))#.61 until here
len_word_q1 = pd.Series(texts_1).apply(lambda x: len(str(x).split()))
len_word_q2 = pd.Series(texts_2).apply(lambda x: len(str(x).split())) #.59 until here
common_words = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
#down to 51

In [181]:
hist.history

{'acc': [0.66837198818970756,
  0.71693036624903883,
  0.74021398281277306,
  0.75507680131926191,
  0.76700031050163775,
  0.77608482363576803,
  0.78318781069912502,
  0.78978785855971778,
  0.79588634128970726,
  0.80016544780658316,
  0.80378908435444341,
  0.8070815503427714,
  0.80944096790993347,
  0.81196940593849276,
  0.81417354434117362],
 'loss': [0.43087042183261626,
  0.35572124865343258,
  0.33113729155331856,
  0.31463765397883586,
  0.30252043344387081,
  0.29297803798897532,
  0.28553758301474291,
  0.27866757922398006,
  0.27253165002420082,
  0.26796472785263592,
  0.26437277323537006,
  0.26079637268015798,
  0.25824614342470531,
  0.25541665385506629,
  0.25331244394861546],
 'val_acc': [0.6782012905759377,
  0.75032773483527537,
  0.76729575288785512,
  0.78694748801078029,
  0.78997749110982396,
  0.80067525794201833,
  0.79702688588003667,
  0.80877587899874193,
  0.81188008633925224,
  0.8135002106063256,
  0.81625813137117098,
  0.82098246281658116,
  0.82482

In [167]:
data=pd.DataFrame({"question1":texts_1,"question2":texts_2})

In [175]:
data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
#50.

/home/ubuntu/anaconda3/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [173]:
fuzz_qratio = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
#slowish
#15:38
fuzz_WRatio = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)

KeyboardInterrupt: 

In [145]:


data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

NameError: name 'data' is not defined

In [134]:
lens=[[len(x1),len(x2)] for x1,x2 in zip(sequences_1,sequences_2)]
lens_diff=[[len(x1)-len(x2)] for x1,x2 in zip(sequences_1,sequences_2)]

Exception ignored in: <bound method Booster.__del__ of <xgboost.core.Booster object at 0x7f5652e40978>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 656, in __del__
    _LIB.XGBoosterFree(self.handle)
AttributeError: 'Booster' object has no attribute 'handle'


In [137]:
lens=np.array(lens)
lens_diff=np.array(lens_diff)

In [139]:
lens_diff.shape

(404290, 1)

In [178]:
xgb_train=np.vstack([len_q1.T,len_q2.T,diff_len.T,len_char_q1.T,len_char_q2.T,len_word_q1.T,len_word_q2.T,\
                    common_words.T,data['fuzz_partial_ratio'].T,data['fuzz_partial_token_set_ratio'].T]).T

In [143]:
xgb_train=np.vstack([lens.T,lens_diff.T]).T[idx_train]
xgb_val=np.vstack([lens.T,lens_diff.T]).T[idx_val]

In [83]:
lens_train.shape

(363861, 2)

In [62]:
y_train=labels[idx_train]
y_val=labels[idx_val]

In [179]:
import xgboost as xgb

# Set our parameters for xgboost
params = {'objective':'binary:logistic','eval_metric':'logloss','eta':0.02,'max_depth':4}

d_train = xgb.DMatrix(xgb_train, label=labels)
#d_valid = xgb.DMatrix(xgb_val, label=y_val)

watchlist = [(d_train, 'train') ]#, (d_valid, 'valid')]

bst = xgb.train(params, d_train, 1500, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.688401
Will train until train-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.649606
[20]	train-logloss:0.622312
[30]	train-logloss:0.602355
[40]	train-logloss:0.587363
[50]	train-logloss:0.575881
[60]	train-logloss:0.567014
[70]	train-logloss:0.560045
[80]	train-logloss:0.554515
[90]	train-logloss:0.550027
[100]	train-logloss:0.546349
[110]	train-logloss:0.54337
[120]	train-logloss:0.540882
[130]	train-logloss:0.538731
[140]	train-logloss:0.536916
[150]	train-logloss:0.535325
[160]	train-logloss:0.534012
[170]	train-logloss:0.532795
[180]	train-logloss:0.531731
[190]	train-logloss:0.530754
[200]	train-logloss:0.529972
[210]	train-logloss:0.529249
[220]	train-logloss:0.528646
[230]	train-logloss:0.528064
[240]	train-logloss:0.527571
[250]	train-logloss:0.527082
[260]	train-logloss:0.526511
[270]	train-logloss:0.525945
[280]	train-logloss:0.525371
[290]	train-logloss:0.524833
[300]	train-logloss:0.524356
[310]	train-logloss:0.523976
[320]	train-logloss:0.5

In [64]:
p_train_l1 = bst.predict(d_train)

In [97]:
p_val_l1 = bst.predict(d_valid)

### second level xgboost

In [78]:
preds_trn[:363861,0]

array([ 0.00336859,  0.36761916,  0.93248445, ...,  0.00658101,
        0.61697412,  0.01418427], dtype=float32)

In [106]:
level_2_trn=np.vstack([preds_trn[:363861,0],preds_trn[363861:,0]]).T

In [107]:
level_2_val=np.vstack([preds_val[:40429,0],preds_val[40429:,0]]).T

In [110]:
from sklearn.metrics import log_loss

In [117]:
labels_val.shape,preds_val[:,0].shape

((80858,), (80858,))

In [120]:
labels_val,preds_val[:,0]

(array([0, 0, 0, ..., 0, 1, 0]),
 array([ 0.04384141,  0.09105706,  0.26341006, ...,  0.32462829,
         0.70351142,  0.77972245], dtype=float32))

In [133]:
model.evaluate([data_1_val, data_2_val], labels_val)

80832/80858 [============================>.] - ETA: 0s

[0.39598954485238164, 0.81625813155988192]

In [132]:
log_loss(labels_val[:8000],preds_val[:8000,0])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1662: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


nan

In [108]:
params = {'objective':'binary:logistic','eval_metric':'logloss','eta':0.02,'max_depth':4}

d_train = xgb.DMatrix(level_2_trn, label=y_train)
d_valid = xgb.DMatrix(level_2_val, label=y_val)

watchlist = [(d_train, 'train') , (d_valid, 'valid')]

bst_level2 = xgb.train(params, d_train, 1500, watchlist, early_stopping_rounds=50, verbose_eval=10)

[0]	train-logloss:0.680137	valid-logloss:0.681765
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 50 rounds.
[10]	train-logloss:0.574113	valid-logloss:0.590256
[20]	train-logloss:0.49948	valid-logloss:0.527672
[30]	train-logloss:0.445204	valid-logloss:0.483735
[40]	train-logloss:0.404894	valid-logloss:0.452482
[50]	train-logloss:0.374534	valid-logloss:0.430131
[60]	train-logloss:0.351445	valid-logloss:0.414182
[70]	train-logloss:0.333765	valid-logloss:0.402911
[80]	train-logloss:0.320168	valid-logloss:0.395091
[90]	train-logloss:0.309681	valid-logloss:0.389828
[100]	train-logloss:0.301576	valid-logloss:0.38644
[110]	train-logloss:0.295303	valid-logloss:0.384415
[120]	train-logloss:0.290446	valid-logloss:0.383371
[130]	train-logloss:0.286683	valid-logloss:0.383052
[140]	train-logloss:0.28377	valid-logloss:0.38322
[150]	train-logloss:0.281513	valid-logloss:0.383756
[160]	train-logloss:0.279768	val